# Inhibition

This notebook recreates parts of the ["Neuron" tutorial of Chapter 3 of the CCN textbook](https://grey.colorado.edu/CompCogNeuro/index.php/CCNBook/Sims/Neuron/Neuron). It uses the Leabra framework corresponding to [emergent](https://grey.colorado.edu/emergent) 8.1. 

In [ ]:
import numpy as np

import dotdot
import leabra
import graphs

**Unit, Layer Specs**

In [ ]:
excite_unit_spec = leabra.UnitSpec(adapt_on=False, noisy_act=True, act_thr=0.5, act_gain=100, tau_net=40, 
                                   g_bar_e=1.0, g_bar_l=0.1, g_bar_i=0.40)
inhib_unit_spec  = leabra.UnitSpec(adapt_on=False, noisy_act=True, act_thr=0.4, act_gain=100, tau_net=20, 
                                   g_bar_e=1.0, g_bar_l=0.1, g_bar_i=0.75)
layer_spec   = leabra.LayerSpec(lay_inhib=False)

**Connections Specs**

In [ ]:
ffexcite_spec   = leabra.ConnectionSpec(proj='full', rnd_type='uniform',  rnd_mean=0.25, rnd_var=0.2)
ffinhib_spec    = leabra.ConnectionSpec(proj='full', rnd_type='gaussian', rnd_mean=0.25, rnd_var=0.2)
fbinhib_spec    = leabra.ConnectionSpec(proj='full', rnd_type='uniform',  rnd_mean=0.25, rnd_var=0.2)
# inhibitory connection weights are constant at 0.5.
inhib_spec      = leabra.ConnectionSpec(proj='full', rnd_type='uniform',  rnd_mean=0.5,  rnd_var=0, inhib=True) 
inhibinhib_spec = leabra.ConnectionSpec(proj='full', rnd_type='uniform',  rnd_mean=0.5,  rnd_var=0, inhib=False)

**Network**

In [ ]:
input_layer  = leabra.Layer(10, spec=layer_spec, unit_spec=excite_unit_spec, name='input_layer')
hidden_layer = leabra.Layer(10, spec=layer_spec, unit_spec=excite_unit_spec, name='hidden_layer')
inhib_layer  = leabra.Layer( 2, spec=layer_spec, unit_spec=inhib_unit_spec,  name='inhib_layer')

In [ ]:
# I:Input, H:Hidden, i:inhib 
IH_conn = leabra.Connection(input_layer,  hidden_layer, spec=ffexcite_spec)
Ii_conn = leabra.Connection(input_layer,  inhib_layer,  spec=ffinhib_spec)
Hi_conn = leabra.Connection(hidden_layer, inhib_layer,  spec=fbinhib_spec)
iH_conn = leabra.Connection(inhib_layer,  hidden_layer, spec=inhib_spec)
ii_conn = leabra.Connection(inhib_layer,  inhib_layer,  spec=inhibinhib_spec)

In [ ]:
network_spec = leabra.NetworkSpec(quarter_size=50)
network = leabra.Network(spec=network_spec,
                         layers=[input_layer, hidden_layer, inhib_layer], 
                         connections=[IH_conn, Ii_conn, Hi_conn, iH_conn, ii_conn])

**Input Activity**

In [ ]:
input_activity = 10*[0.0]
for i in [4, 8]:#, 17, 21, 28, 31, 32, 33, 35, 38, 43, 46, 47, 52, 56, 60, 61, 67, 73, 77, 82]:
    input_activity[i] = 0.95

In [ ]:
network.set_inputs({'input_layer': input_activity})

**Running Cycles**

In [ ]:
hidden_acts_avg, inhib_acts_avg = [], []
for i in range(200):
    network.cycle()
    #print(hidden_layer.activities)
    hidden_acts_avg.append(np.mean(hidden_layer.activities))
    inhib_acts_avg.append(np.mean(inhib_layer.activities))

In [ ]:
fig = graphs.line(range(len(hidden_acts_avg)), hidden_acts_avg, color='black', legend='hidden acts avg', show=False)
graphs.line(range(len(inhib_acts_avg)), inhib_acts_avg, fig=fig, color='red', legend='inhib acts avg')